In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
import talib
import pandas_ta as ta
from data_and_research import ac


##  US Stock Data Worklfow
### 1) Split symbols by sector
### 2) Calculate sector ranks
### 3) Store each sector group in a sector table in ArcticDB, e.g. us_equities/sectorname
### 4) Store each stock individually in us_equities/symbol
### 5) Create a sector overview table in ArcticDB that tracks relative strength of sectors (short-term) and (long-term) - ArcticDBs aggregate function can help

In [2]:
univ = ac.get_library('univ', create_if_missing=True)
univ_df = univ.read('us_equities',columns=['Symbol','Name','Sector','Market Cap']).data

symbols = univ_df['Symbol'].unique().tolist()
names = univ_df.Name.to_list()
sectors = univ_df.Sector.to_list()

lib = ac.get_library('us_equities', create_if_missing=True)


In [3]:
# Create a dictionary mapping Symbols to Columns we want to keep
symbol_to_name = dict(zip(univ_df["Symbol"], univ_df["Name"]))
symbol_to_sector = dict(zip(univ_df["Symbol"], univ_df["Sector"]))
symbol_to_mktcap = dict(zip(univ_df["Symbol"], univ_df["Market Cap"]))

In [4]:
# We start with one sector "Health technology" as example 
# and then later generalize it in a loop
# for sector in univ_df['Sector].unique().tolist():

sector = 'Health technology'
sdf = univ_df[univ_df.Sector == sector]
sector_symbols = sdf.Symbol.to_list()

if 'us_equities' not in lib.list_symbols():
    data = yf.download(sector_symbols, group_by="Ticker", period="max", auto_adjust=True)
else:
    print("download the last 1.5 years - depending on needed lookback for our calcs")

df = data.stack(level=0).rename_axis(['Date', 'Symbol']).reset_index(level=1)
df = df.sort_values(by='Symbol',axis='index',kind='stable')

# Insert Name, Sector, MktCap columns
df["Name"] = df["Symbol"].map(symbol_to_name)
df['Sector'] = df['Symbol'].map(symbol_to_sector)
df['Market Cap'] = df['Symbol'].map(symbol_to_mktcap)



[*********************100%***********************]  896 of 896 completed
/var/folders/gb/y5wbxrk56nl43nqmf6f873g40000gn/T/ipykernel_66520/1867711917.py:14: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = data.stack(level=0).rename_axis(['Date', 'Symbol']).reset_index(level=1)


In [7]:
df["20D_EMA"] = df.groupby("Symbol")["Close"].transform(lambda x: ta.ema(x, length=20))
df["50D_EMA"] = df.groupby("Symbol")["Close"].transform(lambda x: ta.ema(x, length=50))
df["200D_EMA"] = df.groupby("Symbol")["Close"].transform(lambda x: ta.ema(x, length=200))

In [9]:
keltner = df.groupby("Symbol").apply(lambda x: ta.kc(x['High'], x['Low'], x['Close'], length=20)).reset_index(level=0, drop=True)
df['KC_Lower'], df['KC_Middle'], df['KC_Upper'] = keltner.iloc[:, 0], keltner.iloc[:, 1], keltner.iloc[:, 2]

/var/folders/gb/y5wbxrk56nl43nqmf6f873g40000gn/T/ipykernel_66520/2588022126.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  keltner = df.groupby("Symbol").apply(lambda x: ta.kc(x['High'], x['Low'], x['Close'], length=20)).reset_index(level=0, drop=True)


ValueError: cannot reindex on an axis with duplicate labels

In [5]:
# Our calculations
df["20D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=20).mean().reset_index(level=0, drop=True)
df["50D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=50).mean().reset_index(level=0, drop=True)
df["200D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=200).mean().reset_index(level=0, drop=True)
df['ATR'] = df.groupby('Symbol').apply(lambda group: talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=20)).reset_index(level=0, drop=True)
df['1M'] = df.groupby('Symbol')['Close'].pct_change(21)
df['3M'] = df.groupby('Symbol')['Close'].pct_change(63)
df['6M'] = df.groupby('Symbol')['Close'].pct_change(126)
df['12M'] = df.groupby('Symbol')['Close'].pct_change(252)
df['RS IBD'] = 2*df['3M']+df['6M']+df['12M'] # IBD Relative Strength =  2x 3M + 1x 6M + 1x 12M
df['RS Rank'] = df.groupby(df.index)['RS IBD'].rank(pct=True)
df["RS Rank 20D MA"] = df.groupby("Symbol")["RS Rank"].rolling(window=20).mean().reset_index(level=0, drop=True)




/var/folders/gb/y5wbxrk56nl43nqmf6f873g40000gn/T/ipykernel_66520/4129725032.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['ATR'] = df.groupby('Symbol').apply(lambda group: talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=20)).reset_index(level=0, drop=True)


In [11]:
ta.kc()

Strategy(name='All', ta=None, description='All the indicators with their default settings. Pandas TA default.', created='Thursday September 12, 2024, NYSE: 7:25:32, Local: 11:25:32 CEST, Day 256/365 (70.00%)')

In [56]:
aapl

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-09-11,221.425003,223.089996,217.889999,222.660004,222.660004,43842596


In [26]:
df = data.stack(level=0).rename_axis(['Date', 'Symbol']).reset_index(level=1)

/var/folders/gb/y5wbxrk56nl43nqmf6f873g40000gn/T/ipykernel_80904/4056709822.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = data.stack(level=0).rename_axis(['Date', 'Symbol']).reset_index(level=1)


In [29]:
df = df.sort_values(by='Symbol',axis='index',kind='stable')

In [41]:
# univ_df = univ_df.set_index('Symbol',drop=True)

In [44]:
# Create a dictionary mapping Symbols to Names
symbol_to_name = dict(zip(univ_df["Symbol"], univ_df["Name"]))
symbol_to_sector = dict(zip(univ_df["Symbol"], univ_df["Sector"]))

# Use the map function to map the Name values based on the Symbol column

df["Name"] = df["Symbol"].map(symbol_to_name)
df['Sector'] = df['Symbol'].map(symbol_to_sector)

In [46]:
df["20D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=20).mean().reset_index(level=0, drop=True)
df["50D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=50).mean().reset_index(level=0, drop=True)
df["200D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=200).mean().reset_index(level=0, drop=True)
df['ATR'] = df.groupby('Symbol').apply(lambda group: talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=20)).reset_index(level=0, drop=True)
df['1M'] = df.groupby('Symbol')['Close'].pct_change(21)
df['3M'] = df.groupby('Symbol')['Close'].pct_change(63)
df['6M'] = df.groupby('Symbol')['Close'].pct_change(126)
df['12M'] = df.groupby('Symbol')['Close'].pct_change(252)
df['RS IBD'] = 2*df['3M']+df['6M']+df['12M'] # IBD Relative Strength =  2x 3M + 1x 6M + 1x 12M
df['RS Rank'] = df.groupby(df.index)['RS IBD'].rank(pct=True)
df["RS Rank 20D MA"] = df.groupby("Symbol")["RS Rank"].rolling(window=20).mean().reset_index(level=0, drop=True)

/var/folders/gb/y5wbxrk56nl43nqmf6f873g40000gn/T/ipykernel_80904/4044305838.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['ATR'] = df.groupby('Symbol').apply(lambda group: talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=14)).reset_index(level=0, drop=True)


Price                     Symbol       Open       High        Low      Close  \
Date                                                                           
2018-02-16 00:00:00+00:00   AADI  69.750000  72.750000  64.199997  67.500000   
2018-02-20 00:00:00+00:00   AADI  65.699997  66.000000  63.750000  66.000000   
2018-02-21 00:00:00+00:00   AADI  60.000000  74.849998  60.000000  71.250000   
2018-02-22 00:00:00+00:00   AADI  67.650002  71.250000  66.750000  68.745003   
2018-02-23 00:00:00+00:00   AADI  65.250000  67.500000  65.250000  67.500000   
...                          ...        ...        ...        ...        ...   
2024-09-05 00:00:00+00:00   ZYXI   7.750000   7.890000   7.740000   7.890000   
2024-09-06 00:00:00+00:00   ZYXI   7.870000   7.890000   7.680000   7.840000   
2024-09-09 00:00:00+00:00   ZYXI   7.800000   7.850000   7.730000   7.830000   
2024-09-10 00:00:00+00:00   ZYXI   7.860000   8.030000   7.740000   8.020000   
2024-09-11 00:00:00+00:00   ZYXI   7.860

In [51]:
df[(df.index == "2024-09-11") & (df['RS Rank 20D MA'] > df['RS Rank'])].sort_values('RS Rank').dropna()

Price,Symbol,Open,High,Low,Close,Volume,Name,Sector,20D_SMA,50D_SMA,200D_SMA,ATR,1M,3M,6M,12M,RS IBD,RS Rank,RS Rank 20D MA
Date,,,,,,,,,,,,,,,,,,,
2024-09-11 00:00:00+00:00,NDRA,0.261900,0.262600,0.237500,0.249600,1464129.0,ENDRA Life Sciences Inc.,Health technology,0.769830,2.686932,31.139483,0.255406,-0.910857,-0.954618,-0.995420,-0.995908,-3.900565,0.001151,0.004839
2024-09-11 00:00:00+00:00,BJDX,0.150000,0.150100,0.142900,0.149400,826199.0,"Bluejay Diagnostics, Inc.",Health technology,0.211420,0.406528,6.416852,0.054399,-0.560588,-0.960266,-0.977170,-0.995596,-3.893297,0.003452,0.003629
2024-09-11 00:00:00+00:00,CDT,0.123300,0.144000,0.125500,0.142500,3420422.0,Conduit Pharmaceuticals Inc.,Health technology,0.144225,0.229670,2.515917,0.031447,-0.161765,-0.947417,-0.958333,-0.986830,-3.839997,0.004603,0.004838
2024-09-11 00:00:00+00:00,VTAK,0.399700,0.449700,0.380000,0.396400,339315.0,"Catheter Precision, Inc.",Health technology,1.249320,1.969728,4.193932,0.286478,-0.724722,-0.927927,-0.931655,-0.930456,-3.717966,0.006904,0.044066
2024-09-11 00:00:00+00:00,AZTR,0.545500,0.580000,0.538000,0.561000,155907.0,Azitra Inc,Health technology,0.641450,1.237440,13.211810,0.184224,-0.175000,-0.875333,-0.921429,-0.990603,-3.662698,0.008055,0.008582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-11 00:00:00+00:00,LBPH,35.619999,35.500000,33.529999,33.599998,304767.0,"Longboard Pharmaceuticals, Inc.",Health technology,35.137500,35.135800,22.413775,2.472830,-0.036697,0.760545,0.768421,4.428110,6.717620,0.985040,0.989805
2024-09-11 00:00:00+00:00,RNA,40.680000,41.000000,39.755001,40.540001,841273.0,"Avidity Biosciences, Inc.",Health technology,42.958250,43.409100,25.891100,2.161399,-0.088579,0.401798,0.786690,5.170472,6.760758,0.986191,0.989287
2024-09-11 00:00:00+00:00,CRBP,52.310001,56.360001,52.689999,52.709999,191709.0,"Corbus Pharmaceuticals Holdings, Inc.",Health technology,56.091000,55.427200,35.567700,4.356099,0.005724,0.047704,0.182369,6.540773,6.818550,0.987342,0.994068


In [35]:
univ_df

,Symbol,Name,Market Cap,Sector
0,A,"Agilent Technologies, Inc.",3.942100e+10,Commercial services
1,AA,Alcoa Corporation,7.370000e+09,Non-energy minerals
2,AACT,Ares Acquisition Corporation II,5.385000e+08,Finance
3,AADI,"Aadi Bioscience, Inc.",4.504500e+07,Health technology
4,AAGR,African Agriculture Holdings Inc.,7.928000e+06,Process industries
...,...,...,...,...
4642,ZVRA,"Zevra Therapeutics, Inc.",3.851620e+08,Health technology
4643,ZVSA,"ZyVersa Therapeutics, Inc.",2.696000e+06,Health technology
4644,ZWS,Zurn Elkay Water Solutions Corporation,5.078000e+09,Producer manufacturing
4645,ZYME,Zymeworks Inc.,8.279040e+08,Health technology


In [5]:
ac

Arctic(config=S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb))